# CS145: Project 2
## Part 2 | Exploring the GitHub Dataset with Colaboratory (60 points)
---

### Notes (read carefully!):

* Be sure you read the instructions on each cell and understand what it is doing before running it.
* Don't forget that if you can always re-download the starter notebook from the course website if you need to.
* You may create new cells to use for testing, debugging, exploring, etc., and this is in fact encouraged!
**Just make sure that the final answer for each question is _in its own cell_ and _clearly indicated_**.
* Colab will not warn you about how many bytes your SQL query will consume.  **Be sure to check on the BigQuery UI first before running queries here!**
* See the assignment handout for submission instructions.
* Have fun!

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Hang Jiang, hjian42*

## Setting Up BigQuery and Dependencies

Run the two cells below (shift + enter) to authenticate your project and import the libraries you'll need. 

Note that you need to fill in the `project_id` variable with the Google Cloud project id you are using for this course.  You can see your project ID by going to https://console.cloud.google.com/cloud-resource-manager.



In [0]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user()
project_id = "rapid-snowfall-218803"

In [0]:
# Some imports you will need
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import pandas as pd
%matplotlib inline
plt.style.use('seaborn-whitegrid')

# Overview

As we saw in class, BigQuery has a massive dataset of GitHub files and statistics, including information about repositories, commits, and file contents.  In this part of the assignment, we will be working with this dataset further.  Don't worry if you are not too familiar with Git and GitHub -- we will explain everything you need to know to complete this part of the assignment.

---
## Notes

**The GitHub dataset available on BigQuery is actually quite massive. *A single query on the "contents" table alone (it is 2.16TB!) can eat up your 1TB allowance for the month AND cut into about 10% of your GCloud credit for the class.***

To make this part of the project more manageable, we have subset the original data.  We have preserved almost all information in the original tables, but we kept only the information on the top 500,000 most "watched" GitHub repos between January, 2016 and the most recent data as of October, 2018.

You can see the tables we will be working with [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2). **Read through the schemas to get familiar with the data**.  Note that some of the tables are still quite large (the contents table is about 500GB), so you should exercise the usual caution when working with them.  Before running queries on this notebook, it's good practice to first set up query limits on your BigQuery account or see how many bytes will be billed on the web UI.

---

## A Super Quick Primer on Git

If you are not very familiar with Git and GitHub, here are some high-level explanations that will give you enough context to get you through this part of the problem:

*  *GitHub*: GitHub is a source-control service provider.  GitHub allows you to collaborate on and keep track of source code in a fairly efficient way.

*  *commit*: A commit can be thought of as a change that is applied to some set of files. i.e., if some set of files is in some state A, you can make changes to A and *commit* your changes to the set of files so that it is now in state B.  A commit is identified by a *hash* of the information in your change (the author of the commit, who actually committed [i.e. applied] the changes to the set of files, the changes themselves, etc.)

* *parent commit*: The commit that came before your current commit.

*   *repo*:  A repo (short for repository) is GitHub's abstraction for a collection of files along with a history of commits on those files. If you have GitHub username "foo" and you make a repository called "data-rocks", your repo name will be "foo/data-rocks".  You can think of a repo's history in terms of its commits.  E.g., "foo/data-rocks" can have the set of "states" A->B->C->D, where each state change (A->B, B->C, C->D) was due to a commit.

*   *branch*:  To keep track of different commit histories, GitHub repos can have branches.  The 'main' branch (i.e. commit history) of the repo is called the 'master' branch.  Say on "foo/data-rocks" we have the commit history A->B->C->D on the master branch.  If someone else comes along and decides to add a cool new feature to "foo/data-rocks", they can create a branch called "cool-new-feature" that branches away from the master branch. All the code from the main branch will be there, but new code added to "cool-new-feature" will not be on the main branch.

*   *ref*: For the purpose of this assignment, you can think of the 'ref' field on the "files" table as referring to the branch in which a file lives in a repository at some point in time.



For the purposes of this question, you don't need to know about the following things in detail:


*   Commit trees
*   The encoding attribute on the commits table

If you want more clarifications about Git and GitHub in order to answer this question, be sure to post on Piazza or come to Office Hours.  In many cases, a quick web search will also help answer your question.

# Section 1 | Getting Familiar with the GitHub Data
---

## Understanding the GitHub Tables

As you saw in lecture, E/R diagrams are a way to convey the structure of a database (a set of tables) in a non-technical way that is still precise enough for us to work with technically.  As it turns out, E/R diagrams will prove to be a useful tool in getting familiar with the structure of our GitHub tables before we even analyze any of its data!

## Question 1: CS145 GitHub tables -->  E/R Diagram (6 points)

Using what you learned in lecture, create an E/R diagram for the data contained in the `cs145-ta:project_part2 github_repo` tables [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2).  For the purposes of this exercise, ignore the `github_repo_readme_contents_cs145` table (we'll be using that one later)

### Notes

* The CS145 GitHub Repo tables may not directly translate into an E/R diagram as described in class.  An important part of this question is for you to analyze the existing tables, think about the relationships between objects in the tables as a whole, and construct a reasonable E/R diagram from that analysis.

* Consider the "author" and "committer" attribute structures a single attribute each.  Importantly, you do not need to include `committer.name`, `committer.email`, etc.  in your E/R diagrams.  Note that the "language" attribute is actually an *array* type. Be sure to account for this in your E/R diagram.

* Your E/R diagram should be an image.  You can hand draw it or generate it with your favorite software.  You can include an image in your Colab Notebook as follows:



> 1. Add the image to your own Google Drive.  
> 2. Get a shareable link for your image, and note the "ID" field of the URL.
> > Your link will look something like "`https://drive.google.com/open?id=<some ID>`"
> 3. Add the following markup to your cell `![](https://drive.google.com/uc?export=view&id=<your image ID>)`
> 4. Run the cell


---
- ![ERM  img](https://drive.google.com/uc?export=view&id=1KYOmEtL5czOdhQdSbSJGFa03B-DFWSNo)
---

## Question 2: Explain your E/R Diagram (4 points)

In a small paragraph, explain your E/R diagram.  You should cover at least:

*   what your entities are
*   what the relationships between those entities are (including whether they are one-to-many, many-to-one, etc.)
*   what your keys are for each entity

You should also include a quick explanation of how you determined the overall structure of your chart.

---
- My entities are repo, file and commits
- repo includes files and commits change repos; The relationship between repo and commits is many to one, the relationship between repo and file is one to many. 
- The key for repo is irepo_name, the key for file is id, the key for commits is commit

- I first go through the all the tables and try to find relations. I first find commits can means changing a set of files, also means change the repo, and one commit only makes change to one repo but repo can be changed by different commit.  And repo includes a set of files,  a repo can include not only one file while a file can also belong to many repos. 

---

## Question 3: Translate your E/R Diagram to a Set of Schemas (5 points)

Provide a set of schemas that translate directly to the E/R Diagram you generated in Question 2.  These schemas should not be the same as the schemas in the CS145 GitHub data tables. 

Be sure to specify in your schemas:
1. the ***name*** of each attribute (you don't need to worry about types)
2. the ***key for each schema***
3. ***Foreign keys in your schemas*** and what table they reference. 

---
repo:  
{irepo_name, language, lisence, watch_count}  
key: irepo_name

file:  
{id, size, content, binary, copies, path, ref, irepo_name}  
key: id  
foreign key: irepo_name  

commits:  
{commit, message, tree, parent, author, subject, committer, irepo_name}  
key: commit
foreign key: irepo_name

include relation schema:   
{irepo_name, id}  
key: irepo_name and id

change relation schema:  
{commit, irepo_name}  
key: commit and irepo_name  

*Example table for formatting reference:*  

| irepo_name           | language | lisence | watch_count |
|----------------------|----------|---------|-------------|
| gdg-xian/opensourcechromeapps-cn	 | c        | mit     | 3445        |
|  mozilla/geckodriver | c        | mit     | 43          |

| commit                                   | message | tree                    | parent                                    | author | subject                                                              | comitter | irepo_name                       |
|------------------------------------------|---------|-------------------------|-------------------------------------------|--------|----------------------------------------------------------------------|----------|----------------------------------|
| 0dc831b9bca98a0d8dafb00fa7f20b3aef6cab67 |         | asdfdsafdsfsdafdsfsfsda | 2d63ec38f5bb1f598baa003a964805c852a80b33  |        | ALSA: hdspm - AES32: Enable TCO support                              |          | gdg-xian/opensourcechromeapps-cn |
|  ahudsaifhuaisdf3311dsafsafsdfdsfdsafsdf |         | asdfsdafw332fsaf2fwf32f | 4ba69f9ab7df844125898104e854e063b47c26637 |        | ext4: Fixed inode allocator to correctly track a flex_bg's used_dirs |          | mozilla/geckodriver              |

| irepo_name                               | id                  |
|------------------------------------------|---------------------|
|mozilla/geckodriver | asdasdasdafadsf3ef3 |
| gdg-xian/opensourcechromeapps-cn         | a3f3f32afeafe232f32 |

| commit                                   | irepo_name                       |
|------------------------------------------|----------------------------------|
| 0dc831b9bca98a0d8dafb00fa7f20b3aef6cab67 | mozilla/geckodriver              |
| ahudsaifhuaisdf3311dsafsafsdfdsfdsafsdf  | gdg-xian/opensourcechromeapps-cn |


---

## Question 4: Analysis (4 points)

You now have two sets of schemas:  the ones you came up with in Question 3 based on your E/R diagram, and those that we provided for you.  

**In a single paragraph (no more than 100 words), comment on the tradeoffs between the two.  Which do you think is better?**

Use the discussions that we have had in class as a starting point for your analysis.   There is no single correct answer to this question.  Some people abide strongly to their E/R diagrams, while others decide to slice and dice what would have been on an E/R diagram to better fit their needs.


---
Adantages:   
E/R diagram makes it easier to spot who are the entities and the relationship between them and the attributes of each entity. 

Disadvantages:  
However, it will contains redundency and anamly because it can be decomposed according to BCNF. The provided schema, for example, decompose the attributes like language and license into two small table, which make the table smaller and concise. 


---

# Section 2 | Git Visualizing!
---

## Initial Exploration

Now that we understand the dataset we will be working with very well, let's get down to analyzing some of its properties. For the visualizations in this section, feel free to use whatever graphing library you are most comfortable with.  We recommend using one of the following:


*   seaborn (https://seaborn.pydata.org/tutorial.html)
*   matplotlib (https://matplotlib.org/3.0.0/tutorials/index.html)
*   Altair (https://altair-viz.github.io/)
*   Pandas (https://pandas.pydata.org/pandas-docs/stable/visualization.html)


> * **Note**: that you can plot directly from a Pandas DataFrame if you'd like (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)!





### How to BigQuery in Colab

Jupyter notebooks (what Colab notebooks are based on) have a concept called "magics".
If you write the following line at the top of a `Code` cell:

```
%%bigquery --project $project_id variable # this is the key line
SELECT ....
FROM ...
```

That "%%" converts the cell into a SQL cell. The resulting table that is generated is saved into `variable`. You can now use `variable` with your favorite visualization library to generate plots!

Run the following two cells to see this in action.

In [0]:
%%bigquery --project $project_id example

SELECT lrepo_name, watch_count
FROM `cs145-ta.project_part2.github_repos_cs145`
ORDER BY watch_count DESC
LIMIT 10;

In [0]:
example.head()

## Question 5: Count Breakdowns Across Various Fields (9 points)



Let's get out feet wet with this data by creating the following plots:

1.   License distribution across repos
2.   Language distribution across repos
3.   File size distribution across repos
4.   Distribution of files associated with a repo
5.   Number of commits by author and committer across repos
6.   The distribution of the length of commit messages across repos


Note that you will not receive full credit if your charts are  poorly made (i.e. very unclear or unreadable).


### Hints
* Some of these plots will need at least one of their axes to be log-scaled in order to be readable
* For more readable plots, you can use [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html).  A sample size between 1,000 and 10,000 should give you more readable plots.

##  Reminders
* Be careful with your queries!  Don't run `SELECT *` blindly on a table in this Colab notebook since you will not get a warning of how much data the query will consume.  Always how much data a query will consume on the BigQuery UI first -- you are also better off setting a query limit as we described earlier. 
* Don't forget to use the subsetted GitHub tables we provide [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2), not the original ones on BigQuery. 




### a) License distribution (1 point)
#### (x-axis: license type, y-axis: # repos containing that license)


In [7]:
%%bigquery --project $project_id q5a

SELECT license, COUNT(lrepo_name) AS numOfRepos
FROM `cs145-ta.project_part2.github_repo_licenses_cs145`
GROUP BY license

,license,numOfRepos
0,isc,1461
1,mit,99730
2,cc0-1.0,1030
3,epl-1.0,852
4,gpl-2.0,8673
5,gpl-3.0,14906
6,mpl-2.0,1197
7,agpl-3.0,2204
8,lgpl-2.1,1200
9,lgpl-3.0,2057


In [13]:
# YOUR PLOT CODE HERE

alt.Chart(q5a).mark_bar().encode(
    x = 'license',
    y = 'numOfRepos:Q'
)



Chart({
  data:          license  numOfRepos
  0            isc        1461
  1            mit       99730
  2        cc0-1.0        1030
  3        epl-1.0         852
  4        gpl-2.0        8673
  5        gpl-3.0       14906
  6        mpl-2.0        1197
  7       agpl-3.0        2204
  8       lgpl-2.1        1200
  9       lgpl-3.0        2057
  10     unlicense        1498
  11    apache-2.0       28578
  12  artistic-2.0         153
  13  bsd-2-clause        3692
  14  bsd-3-clause        8748,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'license'
    }),
    y: Y({
      shorthand: 'numOfRepos:Q'
    })
  }),
  mark: 'bar'
})

### b) Language distribution (1 point)
#### (x-axis: programming language, y-axis: # repos containing at least one file in that language)

To keep the chart readable, only keep the top 20 languages.

**Hint:** https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays


In [17]:
%%bigquery --project $project_id q5b

SELECT  name, COUNT(name) AS numOfRepo
FROM `cs145-ta.project_part2.github_repo_languages_cs145` ,UNNEST(language)
GROUP BY name
ORDER BY numOfRepo DESC
LIMIT 20

,name,numOfRepo
0,JavaScript,60436
1,Shell,45237
2,HTML,42574
3,Python,36700
4,CSS,35450
5,Makefile,22454
6,Java,21377
7,Ruby,19881
8,C,19390
9,C++,17522


In [19]:
# YOUR PLOT CODE HERE
alt.Chart(q5b).mark_bar().encode(
    alt.Y("numOfRepo:Q", axis=alt.Axis(title="Number of repo")),
    x = 'name',
    
)

Chart({
  data:            name  numOfRepo
  0    JavaScript      60436
  1         Shell      45237
  2          HTML      42574
  3        Python      36700
  4           CSS      35450
  5      Makefile      22454
  6          Java      21377
  7          Ruby      19881
  8             C      19390
  9           C++      17522
  10  Objective-C      13860
  11          PHP      13433
  12           Go       9580
  13           C#       8560
  14    Batchfile       8078
  15        Swift       5966
  16        CMake       5460
  17   TypeScript       4483
  18   Dockerfile       4261
  19         Perl       4012,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'name'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of repo'
      }),
      shorthand: 'numOfRepo:Q'
    })
  }),
  mark: 'bar'
})

### c) File size distribution (1 point)
#### (x-axis: file size, y-axis: # files of that size) 

In [0]:
%%bigquery --project $project_id q5c

SELECT size, COUNT(id) AS num
FROM `cs145-ta.project_part2.github_repo_contents_cs145` 
GROUP BY size

In [59]:
# YOUR PLOT CODE HERE

sampleq5c = q5c.sample(5000)
alt.Chart(sampleq5c).mark_line(size=1).encode(
    alt.Y("num:Q", axis=alt.Axis(title="File numbuer of this size")),
    alt.X("size:Q", axis=alt.Axis(title="Size"), scale=alt.Scale(type='log'))
    
    
)

Chart({
  data:             size   num
  568394    120790     8
  438315    395806     3
  219381    680252     1
  512035    803903     5
  380884    425699     2
  479811    955067     3
  290686    972203     1
  234770   1202712     1
  621050    100976    20
  330975    330144     1
  510520    153572     4
  432385    505979     2
  470425    442701     3
  69653    1853943     1
  704194     21547   233
  212850    463874     1
  534700    267798     6
  557006    131149     7
  69669     518662     1
  493272    183014     4
  633292     81007    26
  458285    188494     3
  322605   2447677     1
  77280    1839351     1
  634838     65140    27
  387599    288058     2
  272837    937428     1
  374648    241884     2
  633867     67115    26
  204818   2419392     1
  ...          ...   ...
  279968   1172790     1
  360664    299982     2
  481104    349171     3
  280744    417789     1
  607954     16873   271
  123143    811392     1
  256555    377624     1
  595721    272300    13
  540420    199911     6
  80997    1050599     1
  586085    126064    11
  383703    271483     2
  10559    2971684     1
  133818   1211758     1
  122463   7859061     1
  625461    147952    22
  605459     98937    15
  194821    301873     1
  125815   1788571     1
  52637    1470632     1
  682275      3932  3947
  271365   6351672     1
  198827  10673625     1
  325958    506822     1
  255019    716316     1
  478825    276684     3
  214997    588936     1
  251401   2396691     1
  191073   3034084     1
  568437     24029   264
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Size'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'size:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'File numbuer of this size'
      }),
      shorthand: 'num:Q'
    })
  }),
  mark: MarkDef({
    size: 1,
    type: 'line'
  })
})

### d) Distribution of files associated with a repo (1 point)
#### (x-axis: # files associated with a repo, y-axis: # repos associated with that many files)

In [0]:
%%bigquery --project $project_id q5d

SELECT numOfFile, COUNT(lrepo_name) AS numOfRepo
FROM(SELECT lrepo_name, COUNT(id) AS numOfFile
FROM `cs145-ta.project_part2.github_repo_files_cs145` 
GROUP BY lrepo_name)
GROUP BY numOfFile

In [15]:
# YOUR PLOT CODE HERE
sampleq5d = q5d.sample(5000)
alt.Chart(sampleq5d).mark_line(size=1).encode(
    alt.Y("numOfRepo:Q", axis=alt.Axis(title="Number of Repo")),
    alt.X("numOfFile:Q", axis=alt.Axis(title="Number of File"), scale=alt.Scale(type='log'))
    
    
)

Chart({
  data:       numOfFile  numOfRepo
  742        8252          1
  222        5271          1
  580        2973          1
  4828        443         35
  1211       1805          1
  3888       1348          5
  2884       3318          2
  2810       1345          2
  1200       3321          1
  1094       5333          1
  875        7872          1
  4927        337         51
  3191       2954          2
  1968       4326          1
  4454        777         12
  799        4229          1
  2744       6914          2
  4961        300         60
  5116         41       1154
  3169       6742          2
  4654        697         20
  4501        818         14
  2919       2069          2
  3667       1859          4
  2493       2632          1
  2991       2805          2
  4720        524         25
  1376      40995          1
  4074       1297          6
  3821       1646          5
  ...         ...        ...
  3066       4126          2
  2293       5158          1
  231       13351          1
  1777       5009          1
  771        2474          1
  3361       4512          3
  2857       2623          2
  858        2326          1
  4727        587         25
  2506       1545          1
  4288       1049          9
  2328       3872          1
  312        8895          1
  1958       5020          1
  4688         46       1046
  1029       9778          1
  3053       1306          2
  87        52342          1
  4061       1617          6
  4904        343         47
  5099        212        120
  3088       6418          2
  1473       8179          1
  433        7321          1
  5108        193        124
  2174       2811          1
  638        3338          1
  2718       1779          2
  397        5763          1
  1178       8146          1
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Number of File'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'numOfFile:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Repo'
      }),
      shorthand: 'numOfRepo:Q'
    })
  }),
  mark: MarkDef({
    size: 1,
    type: 'line'
  })
})

### e) Number of commits by author and committer (1 point)
#### (x-axis: # commits, y-axis: # authors/committers with that many commits)

**Note**: for this chart, plot the curves for author and committers side-by-side for comparison.



In [0]:
%%bigquery --project $project_id q5e_authors

# YOUR QUERY HERE
SELECT COUNT(name) AS numOfauthor, numOfCommit
FROM(SELECT COUNT(commit) AS numOfCommit ,author.name
FROM `cs145-ta.project_part2.github_repo_commits_cs145`
GROUP BY author.name )
GROUP BY numOfCommit 


In [0]:
%%bigquery --project $project_id q5e_committers

# YOUR QUERY HERE
SELECT COUNT(name) AS numOfcommitter, numOfCommit
FROM(SELECT COUNT(commit) AS numOfCommit ,committer.name
FROM `cs145-ta.project_part2.github_repo_commits_cs145`
GROUP BY committer.name )
GROUP BY numOfCommit 


In [66]:
# YOUR PLOT CODE HERE - AUTHORS
sampleq5e_authors = q5e_authors.sample(5000)
alt.Chart(sampleq5e_authors).mark_line(size=1).encode(
    alt.Y("numOfauthor:Q", axis=alt.Axis(title="Number of Author")),
    alt.X("numOfCommit:Q", axis=alt.Axis(title="Number of Commit"), scale=alt.Scale(type='log'))     
)

Chart({
  data:       numOfauthor  numOfCommit
  5734           19         1044
  6725          253          195
  3809            2         3937
  3643            2         4269
  4790            6         2136
  6626          182          252
  4036            3         3616
  3717            2         2447
  6555          140          304
  4886            6         1145
  4682            5         3456
  3837            3         2762
  153             1         3587
  5118            8         1922
  2873            1         3716
  4745            5         2156
  6091           37          690
  1920            1         4532
  5221            9         1880
  4556            5         1500
  5883           24         1309
  1710            1        11525
  4520            4         3675
  2726            1        68278
  4909            6         2907
  1977            1        55134
  4609            5         1858
  86              1         8996
  3438            2         2931
  985             1         4986
  ...           ...          ...
  5837           22          923
  1381            1         3344
  1066            1        17153
  3586            2         3670
  4481            4         3130
  2120            1        23274
  2817            1         6748
  1427            1         7284
  3268            2         4638
  752             1         8144
  6605          163          267
  3876            3         2979
  69              1         4008
  6089           37          715
  274             1         4487
  3370            2         4171
  1304            1         3959
  4657            5         3158
  2243            1         4558
  5967           28          725
  809             1         5735
  3290            2         4226
  2119            1         6411
  3704            2         5782
  5347           11         1299
  3116            2         3173
  1916            1         7258
  6192           48          609
  2554            1         7278
  4949            7         1519
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Number of Commit'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'numOfCommit:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Author'
      }),
      shorthand: 'numOfauthor:Q'
    })
  }),
  mark: MarkDef({
    size: 1,
    type: 'line'
  })
})

In [69]:
# YOUR PLOT CODE HERE - COMMITTERS
sampleq5e_committers = q5e_committers.sample(5000)
alt.Chart(sampleq5e_committers).mark_line(size=1).encode(
    alt.Y("numOfcommitter:Q", axis=alt.Axis(title="Number of Committers")),
    alt.X("numOfCommit:Q", axis=alt.Axis(title="Number of Commit"), scale=alt.Scale(type='log'))     
)

Chart({
  data:       numOfcommitter  numOfCommit
  1499               1        10495
  5186              28          818
  1166               1         4677
  3047               2         7420
  4479               9         1264
  4510               9         1299
  4080               5         1779
  3194               2         1633
  146                1         3551
  5723             129          273
  2252               1         6755
  3864               4         1903
  3141               2         5192
  3661               4         1887
  2928               2         3998
  583                1         5582
  5325              39          577
  495                1         3314
  4048               5         1954
  2267               1        22100
  5149              26          929
  352                1        16055
  3089               2         5125
  1725               1         4945
  2936               2         2356
  794                1         4601
  5704             124          297
  4234               6         2010
  2119               1        77300
  3680               4         2066
  ...              ...          ...
  3727               4         2524
  5521              66          416
  3410               3         5260
  4033               5         3409
  4009               5         2322
  5867             481          122
  386                1        13169
  3076               2         2768
  86                 1        14281
  4872              15          997
  4483               9         1365
  4638              11         1550
  41                 1        21070
  2017               1         4412
  3981               5         2608
  2285               1         7029
  778                1         5197
  2432               2         7664
  12                 1        91355
  2949               2         1929
  3086               2         1878
  3494               3         2269
  4674              12         1094
  3562               3         4343
  5612            2648           29
  2815               2         4839
  2974               2         3272
  5040              21          853
  3350               3         4360
  2027               1         7517
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Number of Commit'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'numOfCommit:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Committers'
      }),
      shorthand: 'numOfcommitter:Q'
    })
  }),
  mark: MarkDef({
    size: 1,
    type: 'line'
  })
})

### f) The distribution of the length of commit messages (1 point)
#### (x-axis: length of the commit message, y-axis: # commits with that length)

**Note**: The query for this plot may use ~30GB of data.

In [0]:
%%bigquery --project $project_id q5f

# YOUR QUERY HERE
SELECT COUNT(commit) AS numOfCommit ,length(message) AS len
FROM `cs145-ta.project_part2.github_repo_commits_cs145`
GROUP BY length(message)

In [75]:
# YOUR PLOT CODE HERE
sampleq5f = q5f.sample(5000)
alt.Chart(sampleq5f).mark_line(size=1).encode(
    alt.Y("numOfCommit:Q", axis=alt.Axis(title="Number of Commits")),
    alt.X("len:Q", axis=alt.Axis(title="Length of Message"), scale=alt.Scale(type='log'))     
)

Chart({
  data:        numOfCommit     len
  3700             1    6606
  13594          225    2769
  6628             3   13471
  13137          451    2282
  796              1    8409
  1505             1   20189
  3725             1   22286
  10523         5957    1089
  4566             2   35596
  9604           293    3433
  8831            19    4051
  4359             1   28289
  1022             1   19023
  4546             2    5116
  6306             3    8236
  10115           54    4386
  2488             1   18091
  10969           84    6676
  758              1    7510
  13051          190    2971
  12723         8105     986
  3666             1  145071
  9726         68648     445
  2384             1  129049
  129              1   36882
  10561           71    5912
  9715           296    2921
  7464             5   16284
  6134             3    7885
  2548             1   18743
  ...            ...     ...
  4005             1   20741
  9359          2591    1497
  4716             2   12596
  12023          383    2750
  6869             4    6773
  10850           81   10762
  13153          196    4177
  12099          899    1803
  2193             1   29367
  12236         1675    1838
  7904             7    5710
  8736            17    7837
  4323             1   11999
  1698             1   36521
  2310             1    7216
  5576             2    6958
  7307             5    6946
  5126             2    8477
  13706          489    2471
  640              1   21547
  11838          118    3404
  11768       311154      87
  11643          365    3021
  11974        16764     797
  9119           281    3018
  11049           87    4932
  13934        88826     286
  8646            15    5812
  11019        30037     652
  12858          178    2777
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Length of Message'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'len:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Commits'
      }),
      shorthand: 'numOfCommit:Q'
    })
  }),
  mark: MarkDef({
    size: 1,
    type: 'line'
  })
})

### g) In a single paragraph (no more than 100 words), describe and analyze the charts you just generated.  What are some interesting trends in the data?  Was there anything that you did not expect? (3 points)

---
- MIT license is the most popular license
- JavaScript, Shell, HTML are the top3 languages
- In general, the larger the file is, the less the amount of that size
- Most repos contain about 10 files. More or less than 10 files, there are less repos. 
- The curve of #of committers and # of authors are similar. They both decrease with the amount of commits raise. With the same amount of commits, number of committers often larger than that of authors
- Most messages have the length between 20 to 40. 

- When the data is very large(e.g. the file size of the file amount a repo contains) , the curve will be fluctuant

---

## What Makes a Good Repo?

Given that we have some interesting data at our disposal, let's try to answer the question: what makes a good GitHub repo? For our purposes, a "good" repo is simply a repo with a high watch count; this refers to how many people are following the repo for updates.

To begin, let's see if any of the features *we've already explored* give us any good answers.

## Question 6: Using What We've Worked With (17 points)

Create plots for the following features in a repo and how they relate to that repo's watch count:

1.  License type
2.  Languages
3.  Average file size in a repo
4.  File count in a repo
5.  The number of "power" committers or authors a repo has
6. Average message length of commits on a repo


### a) License type (2 points)

In [0]:
%%bigquery --project $project_id q6a

# YOUR QUERY HERE
SELECT license, AVG(watch_count) AS watch_count
FROM `cs145-ta.project_part2.github_repo_licenses_cs145`c1 , `cs145-ta.project_part2.github_repos_cs145`c2
WHERE c1.lrepo_name = c2.lrepo_name
GROUP BY license

In [80]:
# YOUR PLOT CODE HERE
alt.Chart(q6a).mark_bar().encode(
    alt.Y("watch_count:Q", axis=alt.Axis(title="Average Number of Watch")),
    x = 'license',
    
)

Chart({
  data:          license  watch_count
  0   bsd-3-clause   206.054184
  1            mit   216.790534
  2     apache-2.0   271.957415
  3        cc0-1.0   475.604854
  4        gpl-3.0   153.551523
  5      unlicense   153.594793
  6   bsd-2-clause   179.252438
  7       lgpl-3.0   133.444823
  8        gpl-2.0   120.958838
  9       agpl-3.0   173.819419
  10       mpl-2.0   218.125313
  11      lgpl-2.1   147.007500
  12           isc   134.072553
  13       epl-1.0   102.212441
  14  artistic-2.0   110.666667,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'license'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      shorthand: 'watch_count:Q'
    })
  }),
  mark: 'bar'
})

### b) Languages (2 points)



In [0]:
%%bigquery --project $project_id q6b

# YOUR QUERY HERE
SELECT name, AVG(watch_count) AS watch_count
FROM `cs145-ta.project_part2.github_repo_languages_cs145`c1, UNNEST(language) c2 , `cs145-ta.project_part2.github_repos_cs145`c3
WHERE c1.lrepo_name = c3.lrepo_name
GROUP BY name



In [17]:
# YOUR PLOT CODE HERE
alt.Chart(q6b).mark_bar().encode(
    alt.Y("watch_count:Q", axis=alt.Axis(title="Average Number of Watch")),
    x = 'name',
    
)

Chart({
  data:                name  watch_count
  0         PureBasic  2848.620000
  1            Rascal  2509.727273
  2            HiveQL  2128.428571
  3               LFE  2124.000000
  4          Solidity  2114.611111
  5            WebIDL  1854.857143
  6           Logtalk  1829.666667
  7          IGOR Pro  1817.666667
  8               TLA  1705.166667
  9         Ballerina  1586.500000
  10              Boo  1581.461538
  11             LLVM  1514.592920
  12            Genie  1484.000000
  13      WebAssembly  1440.900000
  14            Smali  1318.625000
  15           Chapel  1176.000000
  16            Fancy   952.666667
  17                q   929.421053
  18      Cap'n Proto   867.828571
  19              Opa   842.000000
  20      AngelScript   831.515625
  21       Dockerfile   741.264727
  22               P4   701.777778
  23              IDL   670.962025
  24        Brainfuck   659.733333
  25          Nearley   658.000000
  26           DTrace   646.560976
  27             Lean   639.785714
  28            COBOL   634.466667
  29              Vue   623.528718
  ..              ...          ...
  313           Alloy    31.000000
  314           JFlex    31.000000
  315         LabVIEW    30.866667
  316         FORTRAN    29.835443
  317          RUNOFF    28.000000
  318        Bluespec    28.000000
  319   Filebench WML    27.333333
  320           Frege    27.111111
  321          Delphi    27.000000
  322              Ox    27.000000
  323               J    26.666667
  324              EQ    26.250000
  325     DCPU-16 ASM    25.000000
  326             SQL    23.142857
  327          LookML    23.000000
  328            Ioke    23.000000
  329      PogoScript    22.500000
  330             NCL    22.333333
  331          XPages    19.000000
  332             ATS    18.666667
  333             ECL    18.000000
  334        PicoLisp    17.500000
  335             KRL    17.000000
  336            Xojo    16.500000
  337          Ren'Py    16.000000
  338            wisp    15.500000
  339          Monkey    15.000000
  340  Graphviz (DOT)    15.000000
  341            Shen    15.000000
  342       REALbasic    12.000000
  
  [343 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'name'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      shorthand: 'watch_count:Q'
    })
  }),
  mark: 'bar'
})

### c) Average file size in a repo (2 points)
**Note:** For this question, you may use the `github_repo_readme_contents_cs145` table instead of the full contents table.


In [0]:
%%bigquery --project $project_id q6c

# YOUR QUERY HERE
SELECT AVG(watch_count) AS averageWatchCount, averageFileSize
FROM 
(SELECT ROUND(AVG(size),0) AS averageFileSize, c2.lrepo_name
FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145`c1,
     `cs145-ta.project_part2.github_repo_files_cs145` c2     
WHERE c1.id = c2.id 
GROUP BY c2.lrepo_name) c3,
`cs145-ta.project_part2.github_repos_cs145` c4
WHERE c3.lrepo_name = c4.lrepo_name
GROUP BY averageFileSize

In [113]:
# YOUR PLOT CODE HERE
sampleq6c = q6c.sample(5000)
alt.Chart(sampleq6c).mark_circle(size=10).encode(
    alt.Y("averageWatchCount:Q", axis=alt.Axis(title="Average Number of Watch"),scale= alt.Scale(type="log")),
    alt.X("averageFileSize:Q", axis=alt.Axis(title="Average File Size"),scale= alt.Scale(type="log"))     
)

Chart({
  data:        averageWatchCount  averageFileSize
  6144          261.000000          17889.0
  10547         150.000000           9568.0
  12753         143.051282            410.0
  12064         113.250000          11633.0
  19706          80.511628            239.0
  18765         149.094340             91.0
  18718         109.444444           5469.0
  16182          36.040000           1399.0
  10711         380.000000          34740.0
  11525         167.250000           9629.0
  10191          71.000000          11239.0
  2497           32.000000          21944.0
  9756          315.000000          19375.0
  6732          454.500000           8781.0
  4606          225.000000           8725.0
  2909           44.000000          13497.0
  7464           37.000000          40224.0
  15760         241.714286           1422.0
  11791         399.500000           9435.0
  11304         151.000000          16738.0
  5594          101.000000          10758.0
  16317          71.384615           1931.0
  11070          91.375000           6662.0
  7023         1630.000000          26748.0
  4668          354.000000          35648.0
  4048         1026.000000          38978.0
  1964           26.000000          11029.0
  19498         136.933333           3125.0
  13373         147.600000           9268.0
  17171          88.666667            945.0
  ...                  ...              ...
  11628         151.500000          24800.0
  1885           25.000000          16038.0
  2053           27.000000          15897.0
  509            15.000000          32365.0
  12062          89.250000           2952.0
  7748           47.000000          22765.0
  19632          76.765957           1313.0
  542            15.000000          30363.0
  19283         185.745098             23.0
  15343         138.586957           1724.0
  15540         398.400000           9283.0
  6691         1049.000000          14645.0
  14695         162.666667          14898.0
  4421           84.500000          13550.0
  19327         156.678571           2072.0
  1948           26.000000          36715.0
  1823           25.000000          14652.0
  14094          63.739130           2044.0
  17826          53.333333           5970.0
  12008         145.750000           1595.0
  13644         212.806452           1792.0
  14887         174.166667            872.0
  2826           40.000000          17497.0
  7632           43.000000          49307.0
  15516          59.800000           8087.0
  9945           22.750000           7300.0
  3191           54.000000         390865.0
  11510        5861.000000         113414.0
  18153         232.440000            347.0
  18395        2381.714286           4909.0
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average File Size'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'averageFileSize:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'averageWatchCount:Q'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'circle'
  })
})

### d) File count in a repo (2 points)

In [0]:
%%bigquery --project $project_id q6d

# YOUR QUERY HERE
SELECT AVG(watch_count) AS avgWatchCount, numOfFiles
FROM(SELECT COUNT(id) AS numOfFiles, lrepo_name
FROM `cs145-ta.project_part2.github_repo_files_cs145` 
GROUP BY lrepo_name)c1, `cs145-ta.project_part2.github_repos_cs145` c2
WHERE c1.lrepo_name = c2.lrepo_name
GROUP BY numOfFiles

In [26]:
# YOUR PLOT CODE HERE
sampleq6d = q6d.sample(5000)
alt.Chart(sampleq6d).mark_circle(size=10).encode(
    alt.Y("avgWatchCount:Q", axis=alt.Axis(title="Average Number of Watch"),scale=alt.Scale(type = "log")),
    alt.X("numOfFiles:Q", axis=alt.Axis(title="Number of Files in a repo"),scale=alt.Scale(type = "log"))  
    
)

Chart({
  data:       avgWatchCount  numOfFiles
  2966     165.000000       13515
  4794     311.666667        2685
  2227     716.500000        1692
  1502    1219.500000        3310
  3928      57.142857         815
  3278     134.000000       13199
  4271     564.666667        2970
  277       15.000000        2944
  3208      83.000000        5059
  4245     131.166667        1380
  4846     245.333333        3792
  2602      53.000000        8362
  2859    2019.000000        2137
  5065     227.144309          81
  1645     258.000000        2987
  3214      83.000000        7673
  4716     611.333333        1038
  2663      63.000000       56328
  4170     230.666667        1027
  3697   34068.000000        4834
  4238     202.666667        1764
  5200     196.172589         150
  1379     124.000000       14891
  4803     196.833333        1192
  665       22.000000        4137
  328       16.000000        1627
  3374     364.750000        1492
  129       14.000000       12358
  3663    2559.000000        3502
  671       22.000000        5996
  ...             ...         ...
  3243     103.000000        8730
  87        13.000000        7811
  1685     644.750000        2160
  3655    1278.000000       11593
  1085      38.000000        9957
  71        13.000000        2538
  3893     170.255396          43
  1519     321.000000       15384
  1402     160.000000        3817
  2829    1008.000000       14643
  1738     355.000000        2414
  3603     115.750000        1688
  515       19.000000        4232
  2289     471.000000        6381
  4418     132.714286        1953
  1230      58.000000        3569
  2306    1757.000000        5919
  5077     130.428571        1396
  1556    1798.000000        3334
  1168      48.000000        8872
  4858    1140.666667        1190
  2789     164.000000        3306
  1393     144.000000        1557
  4420     247.714286        1458
  113       14.000000        4526
  3556    1272.000000        8443
  2752     114.000000        3421
  1637     241.000000        5544
  1364     116.000000       11262
  5060      98.294118         635
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Number of Files in a repo'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'numOfFiles:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgWatchCount:Q'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'circle'
  })
})

### e) The number of "power" committers or authors a repo has (3 points)

 **Definition:** a "power" committer or author is an account that has committed/authored 1,000 or more commits.

In [0]:
%%bigquery --project $project_id q6e_power_committers

# YOUR QUERY HERE
 SELECT
  AVG(watch_count) AS watch_count,
  repo_count
FROM (
  SELECT
    lrepo_name,
    COUNT(DISTINCT(committer.name)) AS repo_count
  FROM (
    SELECT
      committer.name,
      COUNT(commit) AS Times
    FROM
      `cs145-ta.project_part2.github_repo_commits_cs145`
    GROUP BY
      committer.name
    HAVING
      COUNT(commit) > 1000) c1,
    `cs145-ta.project_part2.github_repo_commits_cs145` c2
  WHERE
    c1.name = c2.committer.name
  GROUP BY
    lrepo_name)c3,
  `cs145-ta.project_part2.github_repos_cs145`c4
WHERE
  c3.lrepo_name = c4.lrepo_name
GROUP BY
  repo_count

In [9]:
# YOUR PLOT CODE HERE

alt.Chart(q6e_power_committers).mark_circle(size=40).encode(
    alt.Y("watch_count:Q", axis=alt.Axis(title="Average Number of Watch"),scale=alt.Scale(type = "log")),
    alt.X("repo_count:Q", axis=alt.Axis(title="Power committers a repo has"),scale=alt.Scale(type = "log"))  
    
)

Chart({
  data:      watch_count  repo_count
  0      13.000000         407
  1      13.000000         409
  2      13.000000         445
  3      13.000000         377
  4      13.000000         272
  5      13.000000         199
  6      13.000000         437
  7      14.000000         507
  8      14.000000         225
  9      14.000000         392
  10     15.000000         402
  11     15.000000         400
  12     15.000000         486
  13     15.000000         431
  14     15.500000         383
  15     16.000000         481
  16     16.000000         113
  17     16.000000         274
  18     18.000000         174
  19     18.000000          91
  20     19.000000         485
  21     20.000000         282
  22     21.000000         240
  23     21.000000         435
  24     21.000000         454
  25     22.000000         366
  26     22.000000         390
  27     23.000000         376
  28     23.000000         154
  29     25.000000         483
  ..           ...         ...
  165  1239.971609          13
  166  3256.333333          50
  167   855.913507           9
  168    84.666667         105
  169  2218.333333          52
  170   716.833333          68
  171   164.333333         104
  172    49.333333         205
  173   823.333333          66
  174   231.333333          65
  175   287.666667         102
  176  5707.666667          55
  177  1768.876289          21
  178  1802.736842          36
  179   283.212119           3
  180  1046.357143          30
  181   712.063283           7
  182   790.819601           8
  183  1382.590604          18
  184  9654.600000          60
  185   201.666954           2
  186  8767.804348          26
  187   328.857143          76
  188  1131.428571          46
  189   863.837545          14
  190  1742.978261          27
  191  1101.594937          12
  192  2556.203704          25
  193  1735.000000          51
  194  2401.000000          32
  
  [195 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Power committers a repo has'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'repo_count:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'watch_count:Q'
    })
  }),
  mark: MarkDef({
    size: 40,
    type: 'circle'
  })
})

In [0]:
%%bigquery --project $project_id q6e_power_authors

# YOUR QUERY HERE
 SELECT
  AVG(watch_count) AS watch_count,
  repo_count
FROM (
  SELECT
    lrepo_name,
    COUNT(DISTINCT(author.name)) AS repo_count
  FROM (
    SELECT
      author.name,
      COUNT(commit) AS Times
    FROM
      `cs145-ta.project_part2.github_repo_commits_cs145`
    GROUP BY
      author.name
    HAVING
      COUNT(commit) > 1000) c1,
    `cs145-ta.project_part2.github_repo_commits_cs145` c2
  WHERE
    c1.name = c2.author.name
  GROUP BY
    lrepo_name)c3,
  `cs145-ta.project_part2.github_repos_cs145`c4
WHERE
  c3.lrepo_name = c4.lrepo_name
GROUP BY
  repo_count

In [10]:
# YOUR PLOT CODE HERE

alt.Chart(q6e_power_authors).mark_circle(size=50).encode(
    alt.Y("watch_count:Q", axis=alt.Axis(title="Average Number of Watch"),scale=alt.Scale(type = "log")),
    alt.X("repo_count:Q", axis=alt.Axis(title="Power authors a repo has"),scale=alt.Scale(type = "log"))  
    
)

Chart({
  data:       watch_count  repo_count
  0       13.000000        3168
  1       13.000000         102
  2       13.000000        3186
  3       13.000000        3373
  4       13.000000        3017
  5       13.000000         329
  6       13.000000        2932
  7       13.000000        3255
  8       13.500000        3372
  9       14.000000         107
  10      14.000000        3391
  11      14.000000        2712
  12      14.000000        3371
  13      14.000000        3123
  14      15.000000        3136
  15      15.000000        3228
  16      15.000000         154
  17      15.000000        2959
  18      15.000000        3130
  19      15.000000        3245
  20      15.000000         405
  21      15.000000        3014
  22      15.000000        3061
  23      15.000000         217
  24      16.000000         224
  25      16.000000        3173
  26      16.000000         331
  27      16.000000        3005
  28      17.000000        2941
  29      17.000000        1998
  ..            ...         ...
  246   3110.333333          72
  247    503.666667         112
  248   1510.666667          89
  249   1205.668874          15
  250   3567.538462          52
  251   1842.769231          56
  252   1692.604651          31
  253   1673.871795          27
  254    311.370348           3
  255  14303.956522          45
  256   1086.786667          14
  257   2467.882353          51
  258   1678.076923          44
  259   1380.111111          58
  260   1360.510638          29
  261   1988.898876          24
  262   4096.800000          74
  263   1797.303030          37
  264   2811.519231          35
  265   4639.714286          55
  266    439.857143          87
  267    458.857143          59
  268   1436.459459          34
  269    366.952586           4
  270   1625.675676          39
  271   3623.407407          38
  272   1534.861004          16
  273    499.402188           5
  274   1811.754098          26
  275   2809.880000          50
  
  [276 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Power authors a repo has'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'repo_count:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'watch_count:Q'
    })
  }),
  mark: MarkDef({
    size: 50,
    type: 'circle'
  })
})

### f) Average message length of commits on a repo. (3 points)

First make a plot of the average commit length of repositories against the number of repositories with that average commit length. Then, make a plot of how average commit length on a repository correlates to its watch count. Round to nearest byte.


In [0]:
%%bigquery --project $project_id q6f_avg_commit_length

# YOUR QUERY HERE
SELECT  avgLength, COUNT(lrepo_name) AS numOfRepo
FROM
  (SELECT ROUND(AVG(length(message)),0) AS avgLength, lrepo_name
  FROM `cs145-ta.project_part2.github_repo_commits_cs145` 
  GROUP BY lrepo_name)
GROUP BY avgLength


In [63]:
# YOUR PLOT CODE HERE
#sampleq6f_avg_commit_length = q6f_avg_commit_length.sample(5000)
alt.Chart(q6f_avg_commit_length).mark_line(size=10,clip=True).encode(
    alt.Y("numOfRepo:Q", axis=alt.Axis(title="Number of repos")),
    alt.X("avgLength:Q", axis=alt.Axis(title="Average Length"),scale = alt.Scale(domain= [0,400])) 
    
)

Chart({
  data:      avgLength  numOfRepo
  0        446.0          1
  1        439.0          1
  2        664.0          1
  3       3665.0          1
  4       6976.0          1
  5        413.0          1
  6        600.0          1
  7        404.0          1
  8        513.0          1
  9       3556.0          1
  10      1153.0          1
  11       491.0          1
  12       410.0          1
  13      1363.0          1
  14       526.0          1
  15       552.0          1
  16       586.0          1
  17       979.0          1
  18       383.0          1
  19       338.0          1
  20       593.0          1
  21       475.0          1
  22       625.0          1
  23      1324.0          1
  24       904.0          1
  25       694.0          1
  26       469.0          1
  27      1359.0          1
  28      1255.0          1
  29       511.0          1
  ..         ...        ...
  617       44.0       3281
  618       24.0       2771
  619      110.0        214
  620      108.0        214
  621       89.0        471
  622       87.0        472
  623       58.0       2008
  624       34.0       3545
  625       63.0       1501
  626       72.0        994
  627      106.0        227
  628       61.0       1766
  629      107.0        231
  630       85.0        490
  631       16.0       1773
  632       54.0       2286
  633       88.0        496
  634       86.0        497
  635       77.0        753
  636       39.0       3569
  637      104.0        243
  638       57.0       2036
  639      105.0        245
  640       19.0       2044
  641       33.0       3580
  642       60.0       1789
  643       35.0       3581
  644       66.0       1278
  645       67.0       1279
  646       46.0       3071
  
  [647 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average Length'
      }),
      scale: Scale({
        domain: [0, 400]
      }),
      shorthand: 'avgLength:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of repos'
      }),
      shorthand: 'numOfRepo:Q'
    })
  }),
  mark: MarkDef({
    clip: True,
    size: 10,
    type: 'line'
  })
})

In [0]:
%%bigquery --project $project_id q6f_msg_length_watch_count

# YOUR QUERY HERE
SELECT AVG(watch_count) AS avgWatch_count, avgLength
FROM
  (SELECT ROUND(AVG(length(message)),0) AS avgLength, lrepo_name
  FROM `cs145-ta.project_part2.github_repo_commits_cs145` 
  GROUP BY lrepo_name)c1, `cs145-ta.project_part2.github_repos_cs145` c2
WHERE c1.lrepo_name = c2.lrepo_name
GROUP BY avgLength


In [119]:
# YOUR PLOT CODE HERE
#sampleq6f_msg_length_watch_count = q6f_msg_length_watch_count.sample(5000)
alt.Chart(sampleq6f_msg_length_watch_count).mark_circle(size=10).encode(
    alt.Y("avgWatch_count:Q", axis=alt.Axis(title="Average Number of Watch"),scale=alt.Scale(type = "log")),
    alt.X("avgLength:Q", axis=alt.Axis(title="Average Length"),scale=alt.Scale(type = "log"))  
    
)

Chart({
  data:         avgWatch_count   avgLength
  105778       51.000000   46.294408
  138971       17.166667   46.600000
  109754       66.000000   44.536842
  118455      531.000000  100.176996
  123109       26.750000   33.583333
  62657        52.000000   48.048780
  121885      189.500000   12.250000
  62263        50.000000   36.780000
  117027     1249.000000   33.220506
  44589        27.000000   63.560976
  74455       144.500000   63.666667
  106538       53.000000   53.493976
  42268        26.000000   89.005910
  97623        35.000000   34.018519
  35375        22.000000   37.399642
  41128        25.000000   83.386652
  127651      111.000000   13.096154
  27162        19.000000   41.314286
  85089       109.000000   61.724138
  135256      765.000000   69.411429
  32441        21.000000   70.600000
  79237       291.000000   37.254355
  70024       100.000000   39.736655
  19825        17.000000   72.672222
  107439       55.000000    9.075000
  48368        29.000000   39.101695
  118365     1701.000000   49.854271
  2500         13.000000   47.060000
  77243       201.000000   42.977273
  21335        17.000000   11.578947
  ...                ...         ...
  131711      429.000000   50.777046
  131650      397.000000   24.924242
  6058         14.000000   27.155340
  8407         14.000000   40.531100
  23100        18.000000   41.434211
  82887        81.000000   27.214286
  115537      188.000000   63.352174
  42211        26.000000   15.958333
  86549       154.000000   63.690096
  51172        31.000000  134.344435
  6846         14.000000   29.578125
  72468       184.000000   63.088050
  101522       41.000000   51.795455
  107070       55.000000   40.777778
  115935      220.000000   87.185417
  3725         13.000000   29.222222
  31846        21.000000  175.699321
  97919        35.000000   53.050000
  42939        26.000000   55.707093
  138009       34.666667   31.695652
  38629        24.000000   36.448171
  71285       124.000000   21.592233
  106377       53.000000   20.408488
  16094        16.000000   39.141892
  19773        17.000000   45.419811
  18942        17.000000  105.165554
  94776      8444.000000   55.257250
  62517        50.000000   42.973684
  40217        25.000000   51.572650
  46528        28.000000   73.890244
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average Length'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgLength:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgWatch_count:Q'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'circle'
  })
})

### g) Which, if any, of the features we inspected above have the highest correlation with a repo having a high watch count? Does the answer make sense, or does it seem counterintuitive? Explain your answer in a small paragraph, no more than 200 words.  Be sure to cite the charts you generated. (3 points)

---

- The average file size: this one cannot guarantee a high watch count, but between 1,000 and 100,000, it is more likely to have more watch count. It makes sense because many functional file are between that size. Larger or smaller are less likely to be watched. 
- File count in a repo: it has the similar graph as average file size. Some of those who have file count between 1000 to 10,000 have really high watch count. It makes sense because good repos are more likely to have that file count, while file count cannot determine whether the repo is good to have big watch count. 
- Power committers or authors: They are similar and between 30~100, it is more likely to have a large watch count. But that not mean in other conditions they won't have large watch count, just the possibility is smaller. 
- Average message length: Similar to the above one, length between 20 to 80 are more likely to have a bigger file count. Again, length between that size cannot ensure to have big file count. 
- Language: In the language graph, the average count of watch top3 is different to the top3 language used and tha makes me surprise.   



- In general, a good repo(have many watch count) may have the characteristic that the average file size is between 1,000 to 10,000 and file count is between 1000 to 10,000 and have the average message length between 20 to 80. But it doesn't mean a repo with such statistic is a good one. 

---

## What Do Others Have to Say?

At this point we have learned a couple of things about how certain features may or may not impact the popularity of a GitHub repo.  However, we really only looked at features of GitHub repos that we had initially explored when we were getting a feel for the dataset!  There has got to be more things we can inspect than that.

If you do a web search for "how to make my git repo popular," you will find that more than a couple of people suggest investing time in your `README` file.  The `README` usually gives an overview to a GitHub project and may include other information about the codebase such as whether its most recent build passed or how to begin contributing to that repo.  [Here](https://github.com/vuejs/vue/blob/dev/README.md) is an example `README` file for the popular web development framework Vue.js.


## IMPORTANT: Note about Contents Table

Note that the original `github_repo_contents_cs145` table is about half a TB!  In order to save you the pain of using up 500GB of your credits to subset this table into a workable size for this problem, we have done it for you. 

## ***For the rest of this question, be sure that you use the `github_repo_readme_contents_cs145` table ***

## Question 7: Analyzing Features (15 points)

Analyze the following features of a repo's `README` file and how they relate to the popularity of a repository, generating an informative plot for each feature:

1.   Having or not having a `README` file
2.   The length of the `README` file
3.   The number of images (.jpg, .png, .svg) or GIFs (.gif) included in the `README`

Note that this part is intentionally more open ended than usual.  Part of what we are looking for is the thoroughness with which you leverage SQL and your plots to gain insight into these features. 

 **Hint:** BigQuery's SQL string, REGEX, and array functions will be helpful here.
 
 **Hint:** You can develop and test regexes on online testers like https://regex101.com/.

### a) Having or not having a `README` file (4 points)




In [82]:
%%bigquery --project $project_id q7a

# YOUR QUERY HERE
WITH C AS (
SELECT DISTINCT lrepo_name
FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145`c1, 
     `cs145-ta.project_part2.github_repo_files_cs145`c2
WHERE NOT(content IS NULL) AND c1.id = c2.id)
# Some README file is NULL, I exclude them 


SELECT  (lrepo_name IN (SELECT lrepo_name FROM C)) AS have, AVG(watch_count) AS avgWatchCount
FROM `cs145-ta.project_part2.github_repos_cs145` c3
GROUP BY have



,have,avgWatchCount
0,True,212.749868
1,False,118.667617


In [94]:
# YOUR PLOT CODE HERE
alt.Chart(q7a).mark_bar().encode(
    alt.Y("avgWatchCount:Q", axis=alt.Axis(title="Number of Watch Count")),
    alt.X("have", axis=alt.Axis(title="Whether Having README"))  
    
)

Chart({
  data:     have  avgWatchCount
  0   True     212.749868
  1  False     118.667617,
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Whether Having README'
      }),
      shorthand: 'have'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Watch Count'
      }),
      shorthand: 'avgWatchCount:Q'
    })
  }),
  mark: 'bar'
})

### b) The length of the `README` file (2 points)

You may ignore README files with length 0.  

**Note**: If a project has multiple `README` files, you can just take the average size of those files.

In [0]:
%%bigquery --project $project_id q7b

# YOUR QUERY HERE
SELECT avgSize, AVG(watch_count)
FROM(   SELECT lrepo_name, AVG(size) AS avgSize
        FROM (SELECT id, size 
              FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145` 
              WHERE NOT content IS NULL)c1, `cs145-ta.project_part2.github_repo_files_cs145` c2
        WHERE c1.id = c2.id
        GROUP BY lrepo_name)c1, `cs145-ta.project_part2.github_repos_cs145` c3
WHERE c1.lrepo_name = c3.lrepo_name
GROUP BY avgSize

In [121]:
# YOUR PLOT CODE HERE
sampleq7b = q7b.sample(5000)
alt.Chart(sampleq7b).mark_point(size=2).encode(
    alt.Y("f0_:Q", axis=alt.Axis(title="Number of Watch"),scale=alt.Scale(type='log')),
    alt.X("avgSize:Q", axis=alt.Axis(title="Average README File Size"),scale=alt.Scale(type='log'))     
)

Chart({
  data:             avgSize          f0_
  25627   5504.333333    47.000000
  16586  10917.000000  1605.000000
  22833   7482.000000   171.500000
  10140   2949.666667    29.000000
  15645    517.555556   224.000000
  39649   8458.000000    57.666667
  34739   3573.000000   135.000000
  7111     251.400000    22.000000
  24729     55.532110    39.000000
  40573   4755.000000    48.555556
  4099   11869.000000    17.000000
  31547   1960.333333    39.500000
  22400  13622.000000   438.000000
  41911   3195.000000   149.083333
  11045    251.333333    32.000000
  39056  10432.000000   520.666667
  20721  14583.000000   436.000000
  1034     156.615385    13.000000
  504     1684.666667    13.000000
  13447   1242.166667    56.000000
  2154    1293.443069    15.000000
  28302   9853.800000   148.000000
  29295   8562.000000   345.000000
  6716     361.190476    22.000000
  8210    1473.666667    25.000000
  28379   5642.250000   164.000000
  22801   4099.000000   125.750000
  25995    585.666667    51.000000
  28423  11424.000000   180.000000
  43416   3671.000000   173.357143
  ...             ...          ...
  36729   8737.000000   477.250000
  18313   7915.500000  4658.000000
  40361   2055.000000   358.678571
  973       29.857143    13.000000
  22890   3584.666667   295.000000
  22734  11188.000000  1051.000000
  42285   9213.000000    54.333333
  32252   1747.500000    83.000000
  45074    426.000000    98.750000
  33456   7607.000000   268.000000
  23934    112.849315    33.000000
  43090   1053.500000    53.833333
  22371  16543.000000   374.000000
  33020   3003.500000   127.000000
  18602  12932.000000    32.500000
  35131   1175.555556   247.000000
  45038   6153.000000    74.000000
  1465     105.818182    14.000000
  36342   1933.000000   168.937500
  38287   2759.000000    64.571429
  34014    210.500000   166.500000
  40041   3199.000000    52.300000
  20984  12765.000000   745.000000
  15675     55.333333   240.000000
  31831    550.400000    67.000000
  32183   1445.375000    79.000000
  3595   15648.000000    17.000000
  44482   7545.000000    78.714286
  37552   2753.000000   235.260870
  33432   2650.833333   246.000000
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average README File Size'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgSize:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'f0_:Q'
    })
  }),
  mark: MarkDef({
    size: 2,
    type: 'point'
  })
})

### c) The number of images (`.jpg`, `.png`, `.svg`) or GIFs (`.gif`) included in the `README` (5 points)

#### Plot the total count of `.png`, `.jpg`, `.svg`, and `.gif` files included.  Remember you can sample the results on a DataFrame for clearer visualizations.

You can find whether an image/GIF is included in a `README` by searching for the following strings:
*   \!\[{*zero+ characters that are not* ']'}]({*zero+ chars that are not* ')'}.(*png|jpg|svg|gif*))
> Example: `![my_svg](https://foo.io/v/svgs/my_cool_svg.svg)`
* <{*zero+ whitespace*}*img*{*one+ whitespace*}src="{*zero+ chars not *'\"'}.(*png|jpg|svg|gif*)
> Example: `<img src="https://foo.bar/attachments/cat.gif" />`

**Hint**: You should test out your regular expressions on a subset of README files so that you can manually verify they work.

**Note**: If you end up dealing with multiple versions of the same `README.md` file, it's fine to average statistics among them.

In [0]:
%%bigquery --project $project_id q7c

# YOUR QUERY HERE
#[!][[][^]]+[]][(][^)]+[.](png|jpg|svg|gif)[)]
#[<][ ]*(img)[ ]+src=\"[^\"]*[.](png|jpg|svg|gif)

WITH idImgAmount AS(
  SELECT id, COUNT(imgNum) AS imgamount
  FROM (SELECT id, REGEXP_EXTRACT_ALL(content, r"[!][[][^]]*[]][(][^)]*[.](png|jpg|svg|gif)[)]") AS REX
      FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145` 
      UNION ALL
      SELECT id, REGEXP_EXTRACT_ALL(content, r"[<][ ]*img[ ]+src=\"[^\"]*[.](png|jpg|svg|gif)") AS REX
      FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145`)c, UNNEST(c.REX) imgNum
  GROUP BY id)
  
SELECT AVG(watch_count),avgImgAmount   
FROM 
  (SELECT lrepo_name, (SUM(imgamount)/COUNT(DISTINCT c1.id)) AS avgImgAmount
  FROM idImgAmount, `cs145-ta.project_part2.github_repo_files_cs145` c1
  WHERE idImgAmount.id = c1.id
  GROUP BY lrepo_name)c2, `cs145-ta.project_part2.github_repos_cs145` c3
WHERE c2.lrepo_name = c3.lrepo_name
GROUP BY avgImgAmount



In [105]:
# YOUR PLOT CODE HERE

alt.Chart(q7c).mark_point(size=40).encode(
    alt.Y("f0_:Q", axis=alt.Axis(title="Number of Watch"),scale=alt.Scale(type='log')),
    alt.X("avgImgAmount:Q", axis=alt.Axis(title="Average Image Amount"),scale=alt.Scale(type='log'))     
)

Chart({
  data:              f0_  avgImgAmount
  0      12.000000      2.550000
  1      12.000000      1.066667
  2      13.000000      2.275362
  3      13.000000      3.900000
  4      13.000000      1.230769
  5      14.000000      8.666667
  6      14.000000      2.272727
  7      14.000000      7.666667
  8      14.000000      2.382353
  9      15.000000      7.800000
  10     15.000000    126.000000
  11     16.000000      9.952381
  12     16.000000    273.000000
  13     17.000000     11.333333
  14     17.000000      1.952381
  15     17.000000    149.000000
  16     17.000000      3.300000
  17     17.000000     90.000000
  18     18.000000    112.000000
  19     18.000000     13.045455
  20     18.000000      3.833333
  21     18.000000    215.000000
  22     19.000000      4.260870
  23     20.000000      2.285714
  24     20.000000      1.083333
  25     21.000000      6.285714
  26     21.000000      2.083333
  27     22.000000    116.000000
  28     22.000000    991.000000
  29     22.000000      5.857143
  ..           ...           ...
  323   374.431818      1.500000
  324  1199.040404     18.000000
  325  1437.666667      2.666667
  326   701.333333     32.000000
  327  1167.666667      1.727273
  328   552.333333      9.750000
  329  1110.666667     34.000000
  330  5753.666667     74.000000
  331  7067.666667     84.000000
  332    28.666667     17.500000
  333    63.333333      1.428571
  334   334.666667      1.090909
  335  5365.666667     67.000000
  336   630.966667     25.000000
  337   269.985114      2.000000
  338  1027.200000      1.300000
  339   117.200000      2.200000
  340   157.400000      2.600000
  341   817.350000     29.000000
  342   811.181818     35.000000
  343   492.846154      1.250000
  344   889.714286     63.000000
  345  1059.450237     14.000000
  346  1388.675676      4.500000
  347   786.117647      1.400000
  348   207.741935      1.750000
  349   802.904762     20.000000
  350  1513.500000     23.000000
  351  2651.000000     65.000000
  352   697.750000      6.500000
  
  [353 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average Image Amount'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgImgAmount:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'f0_:Q'
    })
  }),
  mark: MarkDef({
    size: 40,
    type: 'point'
  })
})

### d) Would you say that a "good" `README` is correlated with a popular repository, based on the features you studied?  Why or why not?  If you were to analyze more in-depth features on the `README` file for correlation with repo popularity what would they be? (4 points)

---
- In general, those who has README file will have much more average watch count(212 to 118). The distribution of the file size shows that most file have the size between 10 to 100,000, and between 1000 to 10,000 are more likely to have a bigger average watch count. There is no direct relationship between average watch count and average image in a repo. If there are not many images(1~10), it cannot garantee how many times it will be watched. That means it can be watched many times or a few. However, if there are more images, it is more likely to be watched more.  As a conclusion,  REAMME files help a repo to get more watch count, and a good README file(proper size and more images) will increase the possibility of getting more watch count. 

- If I can analyze it further, I think there are other factors. For example, the language of README file, whether the README file is well organized or messed up . 
---

## Optional Extra Credit

The problems below are entirely optional, though you may want to try them if you have time in preparation for the kinds of things you'll do in project 3.

## Extra Credit 1: What other features correlate with a highly watched repo? (6 points extra credit)

We studied only a handful of features that could correlate with a highly watched repo.  Can you find a few more that seem especially promising?  Back your proposed features with data and charts.


---

- The author.time_sec may have relationship with repos. For the reason that a repo lasted longer, it will be watched more.  


---



In [0]:
%%bigquery --project $project_id x1
# YOUR QUERIES HERE
SELECT AVG(watch_count) as avgWatchCount, avgTime
FROM
  (SELECT lrepo_name,AVG(author.time_sec) AS avgTime
  FROM `cs145-ta.project_part2.github_repo_commits_cs145`
  GROUP BY lrepo_name) c1,`cs145-ta.project_part2.github_repos_cs145` c2
WHERE c1.lrepo_name = c2.lrepo_name
GROUP BY avgTime

In [25]:
# YOUR PLOT CODE HERE
samplex1 = x1.sample(5000)
alt.Chart(samplex1).mark_point(size = 10).encode(
    alt.Y("avgWatchCount", axis=alt.Axis(title="Average Number of Watch"),scale=alt.Scale(type='log')),
    alt.X("avgTime:Q", axis=alt.Axis(title="Average Author Time(sec)"), scale = alt.Scale(domain= [1000000000,1600000000]))
    
)

Chart({
  data:         avgWatchCount       avgTime
  39833            19.0  1.401532e+09
  74885            30.0  1.360921e+09
  73241            30.0  1.475017e+09
  165942          119.0  1.460333e+09
  67321            27.0  1.431471e+09
  14130            14.0  1.459145e+09
  34578            18.0  1.474722e+09
  31816            17.0  1.485789e+09
  414              12.0  1.331162e+09
  60372            25.0  1.509940e+09
  125171          454.0  1.511012e+09
  74832            30.0  1.338944e+09
  49126            21.0  1.464526e+09
  172760          255.0  1.424475e+09
  95742            62.0  1.421561e+09
  106317          834.0  1.416107e+09
  120021          292.0  1.420452e+09
  34074            18.0  1.501922e+09
  10596            14.0  1.448974e+09
  117287          117.0  1.447135e+09
  39355            19.0  1.461361e+09
  73787            30.0  1.490769e+09
  138683           47.0  1.460262e+09
  3422             13.0  1.409350e+09
  114420           81.0  1.487885e+09
  101544          116.0  1.308986e+09
  147001           74.0  1.428168e+09
  171235          143.0  1.457559e+09
  105844          353.0  1.411787e+09
  149170           98.0  1.457343e+09
  ...               ...           ...
  170533         7379.0  1.509974e+09
  1958             13.0  1.513135e+09
  100496          104.0  1.475382e+09
  79371            34.0  1.497159e+09
  8890             13.0  1.422759e+09
  15160            14.0  1.474246e+09
  58707            24.0  1.403181e+09
  7681             13.0  1.495865e+09
  69898            28.0  1.473274e+09
  43357            20.0  1.478804e+09
  116126          101.0  1.439482e+09
  61521            25.0  1.488313e+09
  92152            54.0  1.373315e+09
  157259          189.0  1.423839e+09
  85165            40.0  1.409637e+09
  26036            16.0  1.428596e+09
  56858            23.0  1.454013e+09
  171267          143.0  1.459939e+09
  111893           65.0  1.484025e+09
  68587            28.0  1.360604e+09
  149467           98.0  1.478519e+09
  74948            30.0  1.459234e+09
  17446            15.0  1.486057e+09
  103996          240.0  1.411327e+09
  173040          334.0  1.359815e+09
  18071            15.0  1.483446e+09
  46215            20.0  1.441642e+09
  108983          274.0  1.464196e+09
  164300           99.0  1.478641e+09
  112127           65.0  1.374030e+09
  
  [5000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Average Author Time(sec)'
      }),
      scale: Scale({
        domain: [1000000000, 1600000000]
      }),
      shorthand: 'avgTime:Q'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Number of Watch'
      }),
      scale: Scale({
        type: 'log'
      }),
      shorthand: 'avgWatchCount'
    })
  }),
  mark: MarkDef({
    size: 10,
    type: 'point'
  })
})

## Extra Credit 2: Predicting Repo Watch Count (4 points extra credit)

Use linear regression to predict the watch count of a repo given a handful of its features.

BigQuery has built-in Machine Learning functionality that makes it straightforward to do basic Machine Learning tasks like linear and logistic regression.  Use a subset of the features that we've been working with on this problem to predict the watch count of a repo.

You will find the following resources helpful:

*   https://cloud.google.com/bigquery/docs/bigqueryml-intro
* https://cloud.google.com/bigquery/docs/bigqueryml-analyst-start
*   https://cloud.google.com/bigquery/docs/bigqueryml-ncaa

In your final output, be sure we can see the following values:

*   The repo name
*   The values of the features you are using
*   The predicted watch count
*   The actual watch count



In [0]:
# PREDICTION CODE HERE

## Closing Note

Understanding data through visualizations is crucial to any working data scientist or database engineer.  Yet, we only touched the tip of the iceberg with the visualizations we made here.  If you are interested in exploring the subject more, we recommend taking *CS 448B: Data Visualization*. 